In [199]:
import sqlalchemy

In [200]:
# Connect to a local SQLite database. SQLite is a simple file oriented database management "library." 
# in the following we launch a SQLite database stored in the .db file in the local folder.
from pathlib import Path
dbfile = Path("ZMB.db")
#if dbfile.exists():
    #dbfile.unlink()

sqlite_uri = "sqlite:///ZMB.db"
sqlite_engine = sqlalchemy.create_engine(sqlite_uri)

In [201]:
# Check tables
sqlite_engine.table_names()

['main', 'tips']

In [202]:
# Get Ztrax dir
path = Path("/scratch/public/ztrax/Zillow_Assessor/06/ZAsmt")
def get_fields(path, table_name):
    layouts = pd.read_excel(path, sheet=1)
    layout = layouts[layouts['TableName'] == ('ut'+table_name)]
    field_names = layout.FieldName
    return field_names.values, layouts

# Get Main.txt fields
main_fields, layouts = get_fields(Path('/accounts/projects/mzuk/brian-truong/Desktop/ca/Layout.xlsx'), 'Main')
bldg_fields, layouts = get_fields(Path('/accounts/projects/mzuk/brian-truong/Desktop/ca/Layout.xlsx'), 'Building')

# Get CREATE SQL queries for dataset
def get_sql_fields(table_name, sql_name, layouts):
    layout = layouts[layouts['TableName'] == table_name]
    fields = layout['FieldName'] + ' ' + layout['DateType'] + '(' + (layout['MaxLength']).astype(str) + ')'
    if table_name == 'utBuilding':
        fields = fields.str.replace(r'RowID', 'RowID2')
        fields = fields.str.replace(r'BatchID', 'BatchID2')
        fields = fields.str.replace(r'FIPS', 'FIPS2')
        fields = fields.str.replace(r'LoadID', 'LoadID2')
    return ('CREATE TABLE ' + sql_name + '( ' + fields.str.cat(sep=', ') + ');')


# Get UPDATE SQL queries for dataset. This is to solve the unescaped character.
def get_esc(table_name, sql_name, layouts):
    layout = layouts[layouts['TableName'] == table_name]
    fields = layout['FieldName'] + ' ' 
    fields = 'update ' + sql_name + ' set ' + fields + '= REPLACE' + '('
    fields = fields + layout['FieldName'] + ','
    fields = fields + '\'"\', \'%-&-%\')'
    if table_name == 'utBuilding':
        fields = fields.str.replace(r'RowID', 'RowID2')
        fields = fields.str.replace(r'BatchID', 'BatchID2')
        fields = fields.str.replace(r'FIPS', 'FIPS2')
        fields = fields.str.replace(r'LoadID', 'LoadID2')
    print(fields.str.cat(sep='; '))
        
main_sql = get_sql_fields('utMain', 'main', layouts)
#get_esc('utMain', 'main', layouts)
main_sql

'CREATE TABLE main( RowID uniqueidentifier(16), ImportParcelID bigint(8), FIPS char(5), State char(2), County varchar(50), ValueCertDate date(3), ExtractDate varchar(6), Edition smallint(2), ZVendorStndCode char(3), AssessorParcelNumber varchar(50), DupAPN char(2), UnformattedAssessorParcelNumber varchar(50), ParcelSequenceNumber int(4), AlternateParcelNumber varchar(50), OldParcelNumber varchar(50), ParcelNumberTypeStndCode char(1), RecordSourceStndCode char(1), RecordTypeStndCode char(2), ConfidentialRecordFlag char(1), PropertyAddressSourceStndCode char(1), PropertyHouseNumber varchar(13), PropertyHouseNumberExt varchar(10), PropertyStreetPreDirectional char(2), PropertyStreetName varchar(50), PropertyStreetSuffix varchar(6), PropertyStreetPostDirectional char(2), PropertyFullStreetAddress varchar(80), PropertyCity varchar(45), PropertyState char(2), PropertyZip varchar(5), PropertyZip4 char(4), OriginalPropertyFullStreetAddress varchar(100), OriginalPropertyAddressLastline varchar(

In [203]:
get_esc('utMain', 'main', layouts)

update main set RowID = REPLACE(RowID,'"', '%-&-%'); update main set ImportParcelID = REPLACE(ImportParcelID,'"', '%-&-%'); update main set FIPS = REPLACE(FIPS,'"', '%-&-%'); update main set State = REPLACE(State,'"', '%-&-%'); update main set County = REPLACE(County,'"', '%-&-%'); update main set ValueCertDate = REPLACE(ValueCertDate,'"', '%-&-%'); update main set ExtractDate = REPLACE(ExtractDate,'"', '%-&-%'); update main set Edition = REPLACE(Edition,'"', '%-&-%'); update main set ZVendorStndCode = REPLACE(ZVendorStndCode,'"', '%-&-%'); update main set AssessorParcelNumber = REPLACE(AssessorParcelNumber,'"', '%-&-%'); update main set DupAPN = REPLACE(DupAPN,'"', '%-&-%'); update main set UnformattedAssessorParcelNumber = REPLACE(UnformattedAssessorParcelNumber,'"', '%-&-%'); update main set ParcelSequenceNumber = REPLACE(ParcelSequenceNumber,'"', '%-&-%'); update main set AlternateParcelNumber = REPLACE(AlternateParcelNumber,'"', '%-&-%'); update main set OldParcelNumber = REPLACE(

In [211]:
# Get JOIN SQL queries for datasets
def get_sql_join_fields(table_name1, sql_name1, table_name2, sql_name2, layouts):

    layout1 = layouts[layouts['TableName'] == table_name1]
    layout2 = layouts[layouts['TableName'] == table_name2]
    ayout1 = layout1['FieldName']
    ayout2 = layout2['FieldName']
    ayout1 = ayout1[ayout1 != 'PropertyZip4']
    ayout1 = ayout1[ayout1 != 'SourceChkSum']
    ayout2 = ayout2[ayout2 != 'BatchID']
    ayout2 = ayout2[ayout2 != 'FIPS']
    ayout2 = ayout2[ayout2 != 'LoadID']
    ayout2 = ayout2[ayout2 != 'TotalCalculatedBathCount']
    ayout1 = ayout1[ayout1 != 'UnformattedAssessorParcelNumber']
    ayout1 = ayout1[ayout1 != 'AlternateParcelNumber']
    ayout1 = ayout1[ayout1 != 'OldParcelNumber']
    if table_name2 == 'utBuilding':
        ayout2 = ayout2.str.replace(r'RowID', 'RowID2')
    fields1 = layout1['FieldName'] + ' ' + layout1['DateType'] + '(' + (layout1['MaxLength']).astype(str) + ')'
    fields2 = layout2['FieldName'] + ' ' + layout2['DateType'] + '(' + (layout2['MaxLength']).astype(str) + ')'
    fields1 = fields1[fields1 != 'PropertyZip4 char(4)']
    fields1 = fields1[fields1 != 'SourceChkSum int(4)']
    fields2 = fields2[fields2 != 'BatchID int(4)']
    fields2 = fields2[fields2 != 'FIPS char(5)']
    fields2 = fields2[fields2 != 'LoadID bigint(8)']
    fields2 = fields2[fields2 != 'TotalCalculatedBathCount decimal(5,2)']
    fields1 = fields1[fields1 != 'UnformattedAssessorParcelNumber varchar(50)']
    fields1 = fields1[fields1 != 'AlternateParcelNumber varchar(50)']
    fields1 = fields1[fields1 != 'OldParcelNumber varchar(50)']
    fields1_ = layout1['FieldName'] 
    fields2_ = layout2['FieldName']
    fields1_ = fields1_[fields1_ != 'PropertyZip4']
    fields1_ = fields1_[fields1_ != 'SourceChkSum']
    fields2_ = fields2_[fields2_ != 'BatchID']
    fields2_ = fields2_[fields2_ != 'FIPS']
    fields2_ = fields2_[fields2_ != 'LoadID']
    fields2_ = fields2_[fields2_ != 'TotalCalculatedBathCount']
    fields1_ = fields1_[fields1_ != 'UnformattedAssessorParcelNumber']
    fields1_ = fields1_[fields1_ != 'AlternateParcelNumber']
    fields1_ = fields1_[fields1_ != 'OldParcelNumber']
    print(len(ayout1)+len(ayout2))
    print(len(fields1)+len(fields2))
    if table_name2 == 'utBuilding':
        fields2 = fields2.str.replace(r'RowID', 'RowID2')
        fields2 = fields2.str.replace(r'BatchID', 'BatchID2')
        fields2 = fields2.str.replace(r'FIPS', 'FIPS2')
        fields2 = fields2.str.replace(r'LoadID', 'LoadID2')
    print('\'' +fields1_.str.cat(sep='\' ,\'') + ',\'' + fields2_.str.cat(sep='\' ,\'') + '\'')
    print('\n---')
    print('INSERT INTO main_bldg'  + " SELECT " +(('main.'+ayout1)).str.cat(sep=', ') + ', ' +(('bldg.'+ayout2)).str.cat(sep=', ')+" FROM main INNER JOIN bldg WHERE main.RowID = bldg.RowID2;")
    return ('CREATE TABLE ' + sql_name1 + '_' + sql_name2 + '( ' + fields1.str.cat(sep=', ') + ', ' + fields2.str.cat(sep=', ') + ');')
get_sql_join_fields('utMain', 'main', 'utBuilding', 'bldg', layouts)

133
133
'RowID' ,'ImportParcelID' ,'FIPS' ,'State' ,'County' ,'ValueCertDate' ,'ExtractDate' ,'Edition' ,'ZVendorStndCode' ,'AssessorParcelNumber' ,'DupAPN' ,'ParcelSequenceNumber' ,'ParcelNumberTypeStndCode' ,'RecordSourceStndCode' ,'RecordTypeStndCode' ,'ConfidentialRecordFlag' ,'PropertyAddressSourceStndCode' ,'PropertyHouseNumber' ,'PropertyHouseNumberExt' ,'PropertyStreetPreDirectional' ,'PropertyStreetName' ,'PropertyStreetSuffix' ,'PropertyStreetPostDirectional' ,'PropertyFullStreetAddress' ,'PropertyCity' ,'PropertyState' ,'PropertyZip' ,'OriginalPropertyFullStreetAddress' ,'OriginalPropertyAddressLastline' ,'PropertyBuildingNumber' ,'PropertyZoningDescription' ,'PropertyZoningSourceCode' ,'CensusTract' ,'TaxIDNumber' ,'TaxAmount' ,'TaxYear' ,'TaxDelinquencyFlag' ,'TaxDelinquencyAmount' ,'TaxDelinquencyYear' ,'TaxRateCodeArea' ,'LegalLot' ,'LegalLotStndCode' ,'LegalOtherLot' ,'LegalBlock' ,'LegalSubdivisionCode' ,'LegalSubdivisionName' ,'LegalCondoProjectPUDDevName' ,'LegalBuil

'CREATE TABLE main_bldg( RowID uniqueidentifier(16), ImportParcelID bigint(8), FIPS char(5), State char(2), County varchar(50), ValueCertDate date(3), ExtractDate varchar(6), Edition smallint(2), ZVendorStndCode char(3), AssessorParcelNumber varchar(50), DupAPN char(2), ParcelSequenceNumber int(4), ParcelNumberTypeStndCode char(1), RecordSourceStndCode char(1), RecordTypeStndCode char(2), ConfidentialRecordFlag char(1), PropertyAddressSourceStndCode char(1), PropertyHouseNumber varchar(13), PropertyHouseNumberExt varchar(10), PropertyStreetPreDirectional char(2), PropertyStreetName varchar(50), PropertyStreetSuffix varchar(6), PropertyStreetPostDirectional char(2), PropertyFullStreetAddress varchar(80), PropertyCity varchar(45), PropertyState char(2), PropertyZip varchar(5), OriginalPropertyFullStreetAddress varchar(100), OriginalPropertyAddressLastline varchar(100), PropertyBuildingNumber varchar(45), PropertyZoningDescription varchar(75), PropertyZoningSourceCode varchar(75), CensusT

In [197]:
bldg_sql = get_sql_fields('utBuilding', 'bldg', layouts)
bldg_sql

'CREATE TABLE bldg( RowID2 uniqueidentifier(16), NoOfUnits int(4), OccupancyStatusStndCode char(1), PropertyCountyLandUseDescription varchar(45), PropertyCountyLandUseCode varchar(10), PropertyLandUseStndCode varchar(5), PropertyStateLandUseDescription varchar(45), PropertyStateLandUseCode varchar(10), BuildingOrImprovementNumber int(4), BuildingClassStndCode varchar(10), BuildingQualityStndCode char(2), BuildingQualityStndCodeOriginal varchar(10), BuildingConditionStndCode char(2), ArchitecturalStyleStndCode char(2), YearBuilt int(4), EffectiveYearBuilt int(4), YearRemodeled int(4), NoOfStories varchar(10), TotalRooms int(4), TotalBedrooms int(4), TotalKitchens int(4), FullBath int(4), ThreeQuarterBath decimal(5,2), HalfBath decimal(5,2), QuarterBath decimal(5,2), TotalCalculatedBathCount decimal(5,2), TotalActualBathCount decimal(5,2), BathSourceStndCode char(1), TotalBathPlumbingFixtures int(4), RoofCoverStndCode char(3), RoofStructureTypeStndCode char(3), HeatingTypeorSystemStndCod

In [198]:
get_esc('utBuilding', 'bldg', layouts)

update bldg set RowID2 = REPLACE(RowID2,'"', '%-&-%'); update bldg set NoOfUnits = REPLACE(NoOfUnits,'"', '%-&-%'); update bldg set OccupancyStatusStndCode = REPLACE(OccupancyStatusStndCode,'"', '%-&-%'); update bldg set PropertyCountyLandUseDescription = REPLACE(PropertyCountyLandUseDescription,'"', '%-&-%'); update bldg set PropertyCountyLandUseCode = REPLACE(PropertyCountyLandUseCode,'"', '%-&-%'); update bldg set PropertyLandUseStndCode = REPLACE(PropertyLandUseStndCode,'"', '%-&-%'); update bldg set PropertyStateLandUseDescription = REPLACE(PropertyStateLandUseDescription,'"', '%-&-%'); update bldg set PropertyStateLandUseCode = REPLACE(PropertyStateLandUseCode,'"', '%-&-%'); update bldg set BuildingOrImprovementNumber = REPLACE(BuildingOrImprovementNumber,'"', '%-&-%'); update bldg set BuildingClassStndCode = REPLACE(BuildingClassStndCode,'"', '%-&-%'); update bldg set BuildingQualityStndCode = REPLACE(BuildingQualityStndCode,'"', '%-&-%'); update bldg set BuildingQualityStndCode

In [55]:
main_sql

'CREATE TABLE main( RowID uniqueidentifier(16), ImportParcelID bigint(8), FIPS char(5), State char(2), County varchar(50), ValueCertDate date(3), ExtractDate varchar(6), Edition smallint(2), ZVendorStndCode char(3), AssessorParcelNumber varchar(50), DupAPN char(2), UnformattedAssessorParcelNumber varchar(50), ParcelSequenceNumber int(4), AlternateParcelNumber varchar(50), OldParcelNumber varchar(50), ParcelNumberTypeStndCode char(1), RecordSourceStndCode char(1), RecordTypeStndCode char(2), ConfidentialRecordFlag char(1), PropertyAddressSourceStndCode char(1), PropertyHouseNumber varchar(13), PropertyHouseNumberExt varchar(10), PropertyStreetPreDirectional char(2), PropertyStreetName varchar(50), PropertyStreetSuffix varchar(6), PropertyStreetPostDirectional char(2), PropertyFullStreetAddress varchar(80), PropertyCity varchar(45), PropertyState char(2), PropertyZip varchar(5), PropertyZip4 char(4), OriginalPropertyFullStreetAddress varchar(100), OriginalPropertyAddressLastline varchar(

In [78]:
result = sqlite_engine.execute(main_sql)

In [ ]:
# Create County Extracts from ZCountyExtracts.sqlite
"""
sqlite> ATTACH 'ZMainBldg.sqlite' AS zmb;
sqlite> CREATE TABLE los_angeles AS SELECT * FROM zmb.main_bldg WHERE County = 'Los Angeles';
"""

In [79]:
# Check tables
sqlite_engine.table_names()

['main', 'tips']

In [83]:
# length of datasets
length = 147333302
# Works on partitions of the dataset
def work(path, fields, partitions, rows, fn):
    import numpy as np
    div = rows // partitions

    parts = np.arange(0, rows, div)
    for i in parts:
        df = pd.read_csv(path, sep='|', names=fields, skiprows=i, nrows=div, encoding='latin-1')
        print(i)
        fn(df)
        del df
        
def to_sql(df):
    # We can use the pandas.to_sql command to put the data in our SQLlite database:
    df.to_sql("main", sqlite_engine, if_exists='append', index=False)

work(path/'Main.txt', main_fields, 69, length, to_sql)

/usr/local/linux/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2856: DtypeWarning: Columns (5,11,13,15,17,19,20,22,25,44,47,49,52,55,66,74,77,78,83) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


0


/usr/local/linux/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2856: DtypeWarning: Columns (5,11,13,15,17,20,22,25,44,47,49,52,55,66,74,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


2135265


KeyboardInterrupt: 

In [75]:
sql_expr = """
SELECT * FROM main
"""
pd.read_sql(sql_expr, sqlite_engine)

,RowID,ImportParcelID,FIPS,State,County,ValueCertDate,ExtractDate,Edition,ZVendorStndCode,AssessorParcelNumber,...,PropertyAddressCBSACode,PropertyAddressCBSADivisionCode,PropertyAddressMatchType,PropertyAddressDPV,PropertyGeocodeQualityCode,PropertyAddressQualityCode,SubEdition,BatchID,BKFSPID,SourceChkSum
0,083BFB02-2AD5-E511-80C1-3863BB43AC67,8904539,6001,CA,ALAMEDA,None,82010,18,BKF,1-1111-10,...,None,None,None,None,None,None,0,330739,-1,None
1,093BFB02-2AD5-E511-80C1-3863BB43AC67,8904540,6001,CA,ALAMEDA,None,82010,18,BKF,1-1111-11,...,None,None,None,None,None,None,0,330739,-1,None
2,0A3BFB02-2AD5-E511-80C1-3863BB43AC67,8904541,6001,CA,ALAMEDA,None,82010,18,BKF,1-1111-12,...,None,None,None,None,None,None,0,330739,-1,None


In [76]:
sql_expr = """
DROP TABLE main;
"""
sqlite_engine.execute(sql_expr)

In [45]:
# Testing
# defining a toy relation (a.k.a. table), populating it with some toy data, and working through some basic SQL

sql_expr = """
CREATE TABLE students(
    name TEXT uniqueidentifier, 
    gpa FLOAT CHECK (gpa >= 0.0 and gpa <= 4.0), 
    age smallint(2), 
    dept TEXT, 
    gender CHAR(2));
"""
result = sqlite_engine.execute(sql_expr)

In [43]:
# The query returns a sqlalchemy ResultProxy object:
result?

In [ ]:
# We can check to see if the result object contains any data:
result.returns_rows

In [ ]:
sql_expr = """
INSERT INTO students VALUES 
  ('Sergey Brin', 2.8, 40, 'CS', 'M'),
  ('Danah Boyd', 3.9, 35, 'CS', 'F'),
  ('Bill Gates', 1.0, 60, 'CS', 'M'),
  ('Hillary Mason', 4.0, 35, 'DATASCI', 'F'),
  ('Mike Olson', 3.7, 50, 'CS', 'M'),
  ('Mark Zuckerberg', 4.0, 30, 'CS', 'M'),
  ('Sheryl Sandberg', 4.0, 47, 'BUSINESS', 'F'),
  ('Susan Wojcicki', 4.0, 46, 'BUSINESS', 'F'),
  ('Marissa Meyer', 4.0, 45, 'BUSINESS', 'F');

"""
result = sqlite_engine.execute(sql_expr)

In [24]:
sql_expr = """
SELECT * FROM students;
"""
result = sqlite_engine.execute(sql_expr)

In [25]:
# The query returns a persistent connection in the form of a cursor which can be used to read data from the database.
[r for r in result.cursor]

[('Sergey Brin', 2.8, 40, 'CS', 'M'),
 ('Danah Boyd', 3.9, 35, 'CS', 'F'),
 ('Bill Gates', 1.0, 60, 'CS', 'M'),
 ('Hillary Mason', 4.0, 35, 'DATASCI', 'F'),
 ('Mike Olson', 3.7, 50, 'CS', 'M'),
 ('Mark Zuckerberg', 4.0, 30, 'CS', 'M'),
 ('Sheryl Sandberg', 4.0, 47, 'BUSINESS', 'F'),
 ('Susan Wojcicki', 4.0, 46, 'BUSINESS', 'F'),
 ('Marissa Meyer', 4.0, 45, 'BUSINESS', 'F')]

In [26]:
# However, as we read the cursor we advance it and so it can only be used once:
[r for r in result.cursor]

[]

In [27]:
# Use Pandas to query the database. Here we pass the engine (or a connection) into the pandas.read_sql function:
import pandas as pd

sql_expr = """
SELECT * FROM students;
"""
pd.read_sql(sql_expr, sqlite_engine)

,name,gpa,age,dept,gender
0,Sergey Brin,2.8,40,CS,M
1,Danah Boyd,3.9,35,CS,F
2,Bill Gates,1.0,60,CS,M
3,Hillary Mason,4.0,35,DATASCI,F
4,Mike Olson,3.7,50,CS,M
5,Mark Zuckerberg,4.0,30,CS,M
6,Sheryl Sandberg,4.0,47,BUSINESS,F
7,Susan Wojcicki,4.0,46,BUSINESS,F
8,Marissa Meyer,4.0,45,BUSINESS,F


In [28]:
# We can also populate the database using Pandas. In the following we first obtain the Tips dataset from the seaborn visualization library as a dataframe:
import seaborn as sns

tips_df = sns.load_dataset("tips")
tips_df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [31]:
# We can then use the pandas.to_sql command to put the data in our SQLlite database:
#tips_df.to_sql("tips", sqlite_engine)
#We can see that a new table has been added:
sqlite_engine.table_names()

['students', 'tips']

In [32]:
sql_expr = """
SELECT * FROM tips
WHERE size > 4;
"""
pd.read_sql(sql_expr, sqlite_engine)

,index,total_bill,tip,sex,smoker,day,time,size
0,125,29.80,4.20,Female,No,Thur,Lunch,6
1,141,34.30,6.70,Male,No,Thur,Lunch,6
2,142,41.19,5.00,Male,No,Thur,Lunch,5
3,143,27.05,5.00,Female,No,Thur,Lunch,6
4,155,29.85,5.14,Female,No,Sun,Dinner,5
5,156,48.17,5.00,Male,No,Sun,Dinner,6
6,185,20.69,5.00,Male,No,Sun,Dinner,5
7,187,30.46,2.00,Male,Yes,Sun,Dinner,5
8,216,28.15,3.00,Male,Yes,Sat,Dinner,5
